# Strasbourg

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()

%matplotlib inline

In [10]:
from astroplan import Observer
from astroplan import FixedTarget

In [11]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

In [12]:
from astropy.coordinates import get_sun,get_moon

In [13]:
import datetime
import timezonefinder, pytz
from calendar import monthrange
import calendar

In [14]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(12,8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

In [15]:
#EarthLocation.get_site_names()

https://python.hotexamples.com/examples/astropy.time/Time/sidereal_time/python-time-sidereal_time-method-examples.html

In [16]:
def radec2altaz(ra, dec, obstime, lat=None, long=None, debug=False):
    """
    calculates the altitude and azimuth, given an ra, dec, time, and observatory location
    :param ra: right ascension of the target (in degrees)
    :param dec: declination of the target (in degrees)
    :param obstime: an astropy.time.Time object containing the time of the observation.
                    Can also contain the observatory location
    :param lat: The latitude of the observatory. Not needed if given in the obstime object
    :param long: The longitude of the observatory. Not needed if given in the obstime object
    :return:
    """

    if lat is None:
        lat = obstime.lat.degree
    if long is None:
        long = obstime.lon.degree
    obstime = Time(obstime.isot, format='isot', scale='utc', location=(long, lat))

    # Find the number of days since J2000
    j2000 = Time("2000-01-01T12:00:00.0", format='isot', scale='utc')
    dt = (obstime - j2000).value  # number of days since J2000 epoch

    # get the UT time
    tstring = obstime.isot.split("T")[-1]
    segments = tstring.split(":")
    ut = float(segments[0]) + float(segments[1]) / 60.0 + float(segments[2]) / 3600

    # Calculate Local Sidereal Time
    lst = obstime.sidereal_time('mean').deg

    # Calculate the hour angle
    HA = lst - ra
    while HA < 0.0 or HA > 360.0:
        s = -np.sign(HA)
        HA += s * 360.0

    # convert everything to radians
    dec *= np.pi / 180.0
    ra *= np.pi / 180.0
    lat *= np.pi / 180.0
    long *= np.pi / 180.0
    HA *= np.pi / 180.0

    # Calculate the altitude
    alt = np.arcsin(np.sin(dec) * np.sin(lat) + np.cos(dec) * np.cos(lat) * np.cos(HA))

    # calculate the azimuth
    az = np.arccos((np.sin(dec) - np.sin(alt) * np.sin(lat)) / (np.cos(alt) * np.cos(lat)))
    if np.sin(HA) > 0:
        az = 2.0 * np.pi - az

    if debug:
        print("UT: ", ut)
        print("LST: ", lst)
        print("HA: ", HA * 180.0 / np.pi)

    return alt * 180.0 / np.pi, az * 180.0 / np.pi

# Observatoire Astronomique de Strasbourg/Coordinates
- 48.5832° N, 7.7678° E
- Latitude: 48° 34' 59.99" N Longitude: 7° 46' 2.99" E. 

In [17]:
strasbourg_location=EarthLocation(lon=7.7678,lat=48.5832,height=142)

# Find time zone

In [18]:
tf = timezonefinder.TimezoneFinder()

In [19]:
timezone_str = tf.certain_timezone_at(lat=strasbourg_location.lat.degree, lng=strasbourg_location.lon.degree)
timezone_str 

'Europe/Paris'

In [20]:
if timezone_str is None:
    print("Could not determine the time zone")
else:
    # Display the current time in that time zone
    
    #tz = pytz.timezone('Europe/Berlin')
    #berlin_now = datetime.now(tz)
    
    timezone = pytz.timezone(timezone_str)
    timezone0 = pytz.timezone('UTC')
    
    dt = datetime.datetime.utcnow()
    timeStrasbourg_now = datetime.datetime.now(timezone)
    
    print("The UTC Time now %s" % dt)
    print("The actual time in %s is %s" % (timezone_str, dt + timezone.utcoffset(dt)))

The UTC Time now 2021-10-11 15:09:38.263961
The actual time in Europe/Paris is 2021-10-11 17:09:38.263961


In [21]:
utcoffset=timezone.utcoffset(dt).total_seconds()/60./60.
print("The UTC offset in Strasbourg is ",utcoffset," hours")

The UTC offset in Strasbourg is  2.0  hours


# Current time

## Current local time

In [22]:
now = datetime.datetime.now()

In [23]:
now_str=now.strftime("%Y-%m-%dT%H:%M:%S")
now_str

'2021-10-11T17:09:38'

In [24]:
datetime.datetime.now()

datetime.datetime(2021, 10, 11, 17, 9, 38, 310363)

In [26]:
coord_sun_local=get_sun(Time(now_str,location=strasbourg_location))
coord_sun_local

<SkyCoord (GCRS: obstime=2021-10-11T17:09:38.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (196.88340873, -7.17553166, 0.99820111)>

In [27]:
coord_sun_local.ra.hourangle

13.125560581910946

## UTC

In [28]:
utc_now = datetime.datetime.utcnow()

In [29]:
utc_now_str=utc_now.strftime("%Y-%m-%dT%H:%M:%S")
utc_now_str

'2021-10-11T15:10:25'

In [30]:
utc_date_today_str=utc_now.strftime("%Y-%m-%d")
utc_date_today_str

'2021-10-11'

In [31]:
datetime.datetime.utcnow()

datetime.datetime(2021, 10, 11, 15, 10, 39, 62685)

In [32]:
coord_sun_utc=get_sun(Time(utc_now_str,scale="utc"))
coord_sun_utc

<SkyCoord (GCRS: obstime=2021-10-11T15:10:25.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (196.80708758, -7.14435509, 0.9982253)>

In [33]:
coord_sun_utc.ra.hourangle

13.120472505204932

In [34]:
the_sun = FixedTarget(coord=coord_sun_utc, name="Sun")

## Local observation in AltAz frame

In [36]:
frame_strasbourg = AltAz(obstime=utc_now ,location=strasbourg_location)

In [37]:
sunaltazs = get_sun(Time(utc_now_str,scale="utc")).transform_to(frame_strasbourg)

In [38]:
sunaltazs.alt

<Latitude 14.24091129 deg>

In [39]:
sunaltazs.zen

<Angle 75.75908871 deg>

In [40]:
sunaltazs.az

<Longitude 240.99927636 deg>

# Observer

In [41]:
from astroplan import Observer

In [43]:
strasbourg_obs=Observer(location=strasbourg_location, name="Strasbourg")

In [45]:
print(strasbourg_obs.astropy_time_to_datetime(Time(now_str,location=strasbourg_location)))

2021-10-11 17:09:38+00:00


## Is it night ?

In [46]:
strasbourg_obs.is_night(utc_now_str)

False

## Sideral time

In [47]:
lst=strasbourg_obs.local_sidereal_time(Time(utc_now_str))
lst

<Longitude 17.05316631 hourangle>

In [48]:
lst.deg

255.79749462370285

## local solar midnight

In [ ]:
strasbourg_obs.midnight(Time(utc_now_str)).strftime("%Y-%m-%dT%H:%M:%S")

## Alt, AZ

In [ ]:
strasbourg_obs.sun_altaz(Time(utc_now_str))

## Sun Set /sun rize

In [ ]:
strasbourg_obs.sun_rise_time(Time(utc_date_today_str)).strftime("%Y-%m-%dT%H:%M:%S")

In [ ]:
strasbourg_obs.sun_set_time(Time(utc_date_today_str),which="next").strftime("%Y-%m-%dT%H:%M:%S")

## Hour angle

In [ ]:
ha=strasbourg_obs.target_hour_angle(Time(utc_date_today_str), target=the_sun)
ha

In [ ]:
ha.deg

In [ ]:
lst.deg

In [ ]:
ha.deg+the_sun.ra.deg